In [ ]:
import numpy as np
import pandas as pd
import cv2
from PIL import Image, ImageDraw
import os

In [ ]:
mask_path = "masks"
if not (os.path.exists(mask_path)):
    os.makedirs(mask_path)
train_annotations = pd.read_csv("../input/ranzcr-clip-catheter-line-classification/train_annotations.csv")
images_path = "../input/ranzcr-clip-catheter-line-classification/train"
def get_point(chosen):
    points = []
    for i, point in enumerate(train_annotations.iloc[chosen]['data'].split('],')):
        if i==len(train_annotations.iloc[chosen]['data'].split('],'))-1:
            xy = tuple([int(x) for x in point[2:-2].split(', ')])
            points.append(xy)
        else:
            xy = tuple([int(x) for x in point[2:].split(', ')])
            points.append(xy)
    return points

file_names = np.unique(train_annotations['StudyInstanceUID'].values)
for file_name in file_names:
    file_path = images_path + "/" + file_name + ".jpg"
    image = cv2.imread(file_path)
    mask = np.zeros((image.shape[0], image.shape[1]), dtype=np.uint8)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    points = []
    for value in train_annotations.loc[train_annotations['StudyInstanceUID']==file_name].index.values:
        points.append(get_point(value))
    mask = Image.fromarray(mask.astype(np.uint8), "L")
    draw = ImageDraw.Draw(mask)
    for p in points:
        draw.line(p, fill=255, width=15)
    mask.save(mask_path + "/" + file_name + ".jpg")